In [1]:
# ensure classes imported from .py files are dynamically updated
%load_ext autoreload
%autoreload 2

# plot matplots nicely
%matplotlib inline  

In [2]:
import numpy as np
import pandas as pd
import datetime
import xarray as xr
from dateutil.relativedelta import relativedelta
from climate_drought import config, drought_indices as dri, utils

In [83]:
# configure inputs
#aa = config.AnalysisArgs(latitude=36,longitude=-120,start_date='20200116',end_date='20200410')
aa = config.AnalysisArgs(latitude=35,longitude=-120,start_date='20200216',end_date='20200401')
cf = config.Config(outdir= 'output',indir='input',verbose=False)

In [84]:
cdi = dri.CDI(cf,aa)

2023-04-17  08:23:26 INFO No SPI provided.
2023-04-17  08:23:26 INFO Creating new SPI object...
2023-04-17  08:23:26 INFO No SMA provided.
2023-04-17  08:23:26 INFO Creating new SMA object...
2023-04-17  08:23:26 INFO No fAPAR provided.
2023-04-17  08:23:26 INFO Creating new fAPAR object...


In [107]:
df = cdi.process()

2023-04-17  08:42:01 INFO Computing Combined Drought Indicator...
2023-04-17  08:42:01 INFO Completed processing of ERA5 CDI data.


In [108]:
for r in df.iterrows():
    print(r['CDI'])

,tp,spi,smant,fpanv,CDI
2020-01-01,NaN,NaN,NaN,NaN,0
2020-01-11,NaN,NaN,NaN,NaN,0
2020-01-21,NaN,NaN,NaN,NaN,0
2020-02-01,0.000572,-0.029249,NaN,NaN,0
2020-02-11,0.000572,-0.029249,-0.79,0.344,0
2020-02-21,0.000572,-0.029249,-1.40,-0.011,0
2020-03-01,0.000179,-0.983137,-2.19,-0.188,0
2020-03-11,0.000179,-0.983137,-2.31,-0.535,0
2020-03-21,0.000179,-0.983137,-2.33,-0.550,0
2020-04-01,0.003710,-0.540281,-1.67,-0.588,0


In [104]:
(cdi.fpr_shifted < -1).to_numpy().flatten()

array([False, False, False, False, False, False, False, False, False,
       False])

In [82]:
cdi.df_merged

,tp,spi,smant,fpanv
2020-01-01,0.000572,-0.029249,NaN,NaN
2020-01-11,0.000572,-0.029249,NaN,NaN
2020-01-21,0.000572,-0.029249,-0.79,NaN
2020-02-01,0.000179,-0.983137,-1.40,0.344
2020-02-11,0.000179,-0.983137,-2.19,-0.011
2020-02-21,0.000179,-0.983137,-2.31,-0.188
2020-03-01,0.003710,-0.540281,-2.33,-0.535
2020-03-11,0.003710,-0.540281,-1.67,-0.550
2020-03-21,0.003710,-0.540281,-0.29,-0.588
2020-04-01,0.002878,0.469652,0.83,-0.353


In [53]:
sdate_ts = pd.Timestamp(aa.start_date)
nearest_dekad = lambda day:(1 if day<11 else (11 if day<21 else 21))
sdate_sma = pd.Timestamp(year=sdate_ts.year,month=sdate_ts.month,day=nearest_dekad(sdate_ts.day)) - relativedelta(days=20)

In [ ]:
#for sma
# if day = 1, we want month-1, day = 11
# if day = 11, we want month-1, day =21
# if day = 21, we want same month, day=1

In [66]:
nearest_dekad((pd.Timestamp(year=sdate_ts.year,month=sdate_ts.month,day=nearest_dekad(sdate_ts.day)) - relativedelta(days=20)).day)


21

In [50]:

sma_class = SMA_ECMWF if sma_source=='ECMWF' else SMA_GDO
self.sma = sma_class(cfg,aa_new(sdate_sma)) if create_new(sma,sdate_sma,'SMA') else sma

2023-04-17  07:29:06 INFO Computing Combined Drought Indicator...


,tp,spi
2020-01-01,0.000572,-0.029249
2020-01-11,0.000572,-0.029249
2020-01-21,0.000572,-0.029249
2020-02-01,0.000179,-0.983137
2020-02-11,0.000179,-0.983137
2020-02-21,0.000179,-0.983137
2020-03-01,0.003710,-0.540281
2020-03-11,0.003710,-0.540281
2020-03-21,0.003710,-0.540281
2020-04-01,0.002878,0.469652


In [85]:
ds = xr.open_dataset('output/precip_19850101-20221231_monthly_52.5_15.25.nc')
ds

<xarray.Dataset>
Dimensions:    (longitude: 3, latitude: 3, expver: 2, time: 456)
Coordinates:
  * longitude  (longitude) float32 15.15 15.25 15.35
  * latitude   (latitude) float32 52.6 52.5 52.4
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 1985-01-01 1985-02-01 ... 2022-12-01
Data variables:
    tp         (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-03-30 16:19:14 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [104]:
pd.Timestamp('20220202').replace(day=1) - relativedelta(months=1)

Timestamp('2022-01-01 00:00:00')

In [96]:
ds = xr.open_dataset('output/precip_19850101-20221231_monthly_36_-120.nc')
precip = ds.tp.max(['latitude', 'longitude']).load()

In [109]:
time_dekads = utils.dti_dekads(aa.start_date,aa.end_date)
time_dekads

DatetimeIndex(['2020-01-11', '2020-01-21', '2020-02-01', '2020-02-11',
               '2020-02-21', '2020-03-01', '2020-03-11', '2020-03-21',
               '2020-04-01'],
              dtype='datetime64[ns]', freq=None)

In [112]:
def nearest_dekad(dt):
    return pd.Timestamp(year=dt.year,month=dt.month,day=(1 if dt.day<11 else (11 if dt.day<21 else 21)))

In [117]:
nearest_dekad(pd.Timestamp('20200131'))

Timestamp('2020-01-21 00:00:00')

In [118]:
type(spi).__name__

'SPI'

In [111]:
spi.data.index[0].day

1

In [107]:
#spi = dri.SPI(cf,aa)
#spi.download()
spi.data.index[0]

Timestamp('2020-02-01 00:00:00')

In [45]:
sma_edo = dri.SMA_EDO(cf,aa)
fpr = dri.FPAR_EDO(cf,aa)
sma_edo.process()
fpr.process()

2023-04-14  12:31:50 INFO Completed processing of ERA5 soil water data.
2023-04-14  12:31:52 INFO Completed processing of ERA5 fAPAR data.


,fpanv
time,
2020-01-01,0.726
2020-01-11,0.722
2020-01-21,0.521
2020-02-01,0.344
2020-02-11,-0.011
2020-02-21,-0.188
2020-03-01,-0.535
2020-03-11,-0.550
2020-03-21,-0.588


In [46]:
cdi = dri.CDI(cf,aa,spi=spi,sma=sma,fpr=fpr)

In [55]:
shift_date = lambda arr, n: np.pad(arr[:-n],(n,0),'constant',constant_values=(np.nan,))
idx_np = lambda df: df.to_numpy().flatten()

In [63]:
spi_cdi = idx_np(spi.data).repeat(3)
spi_tme = idx_np(spi.data.index).repeat(3)
spi_cdi = shift_date(spi_cdi,3)
spi_tme = shift_date(spi_tme,3)

In [72]:
spi.data.index[0] + datetime.timedelta(days=10)

Timestamp('2020-01-11 00:00:00')

In [53]:


        # ------ SPI ------ :
        df_spi = self.spi.process() if len(self.spi.data) == 0 else self.spi.data
        self.df_spi = df_spi

        # Convert to dekads for consistency with other timeseries
        # Use the same value 3 times a month for each dekad
        

        # Shift backwards by 3 dekads
        
        self.spi_np = spi
        
        # ------ SMA ------
        df_sma = self.sma.process() if len(self.sma.data) == 0 else self.sma.data

        # Shift backwards by 2 dekads
        sma = shift_date(idx_np(df_sma),2)
        self.sma_np = sma

array([   nan,  0.726,  0.722,  0.521,  0.344, -0.011, -0.188, -0.535,
       -0.55 , -0.588], dtype=float32)

In [52]:
cdi.spi_np

array([        nan,         nan,         nan, -0.02924945, -0.02924945,
       -0.02924945, -0.98313719, -0.98313719, -0.98313719, -0.54028095,
       -0.54028095, -0.54028095])

In [47]:
cdi.process()

2023-04-14  12:31:59 INFO Computing Combined Drought Indicator...
2023-04-14  12:32:01 INFO Completed processing of ERA5 soil water data.


ValueError: Length of values (12) does not match length of index (10)

In [67]:
df_spi = cdi.spi.process()
df_sma = cdi.sma.process()
df_fpr = cdi.fpr.process()


2023-04-05  13:22:58 INFO Initiating processing of ERA5 daily data.
2023-04-05  13:22:58 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-05  13:22:58 INFO 

2023-04-05  13:22:58 DEBUG scaled values: 0.002 0.012
2023-04-05  13:22:58 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-05  13:22:58 INFO SPI, 456 values: -3.046 2.676
2023-04-05  13:23:00 INFO Completed processing of ERA5 soil water data.
2023-04-05  13:23:02 INFO Completed processing of ERA5 fAPAR data.


In [85]:
df_sma

,smant
time,
2020-01-01,0.54
2020-01-11,0.19
2020-01-21,-0.08
2020-02-01,-0.05
2020-02-11,-0.14
...,...
2022-09-21,-0.60
2022-10-01,-0.37
2022-10-11,-0.94


In [81]:
len(df_spi.spi.to_numpy().flatten().repeat(3))

108

2023-04-05  12:14:21 INFO Initiating processing of ERA5 daily data.
2023-04-05  12:14:21 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-05  12:14:21 INFO 

2023-04-05  12:14:21 DEBUG scaled values: 0.002 0.012
2023-04-05  12:14:21 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-05  12:14:21 INFO SPI, 456 values: -3.046 2.676
2023-04-05  12:14:23 INFO Completed processing of ERA5 soil water data.
2023-04-05  12:14:24 INFO Completed processing of ERA5 fAPAR data.


In [58]:
cdi.fpr.filelist

['input/fpanv\\fpanv_m_wld_20120101_20121221_t.nc',
 'input/fpanv\\fpanv_m_wld_20130101_20131221_t.nc',
 'input/fpanv\\fpanv_m_wld_20140101_20141221_t.nc',
 'input/fpanv\\fpanv_m_wld_20150101_20151221_t.nc',
 'input/fpanv\\fpanv_m_wld_20160101_20161221_t.nc',
 'input/fpanv\\fpanv_m_wld_20170101_20171221_t.nc',
 'input/fpanv\\fpanv_m_wld_20180101_20181221_t.nc',
 'input/fpanv\\fpanv_m_wld_20190101_20191221_t.nc',
 'input/fpanv\\fpanv_m_wld_20200101_20201221_t.nc',
 'input/fpanv\\fpanv_m_wld_20210101_20211221_t.nc',
 'input/fpanv\\fpanv_m_wld_20220101_20221221_t.nc',
 'input/fpanv\\fpanv_m_wld_20230101_20230311_t.nc']

In [47]:
len(df_spi)

35

In [50]:
df_fpr

,fpanv
time,
2020-01-01,NaN
2020-01-11,NaN
2020-01-21,-1.371
2020-02-01,-1.078
2020-02-11,-0.787
...,...
2022-11-11,NaN
2022-11-21,NaN
2022-12-01,NaN


In [34]:
idx_np = lambda df: df.to_numpy().flatten()
shift_date = lambda arr, n: np.pad(arr[:-n],(n,0),'constant',constant_values=(np.nan,))

# use fAPAR times as CDI delivery dates
time = df_fpr.index

# fapar needs to use previous dekad's values
fpr = shift_date(idx_np(df_fpr),1)

# sma needs to use dekad -2 values
sma = shift_date(idx_np(df_sma),2)

# convert spi to dekads
spi = idx_np(df_spi.spi).repeat(3)[:-2]

# spi needs to be from the previous month
spi = shift_date(spi,3)

# spi is in months, and others are in dekads, so repeat spi 3 times before combining with others
#spi = spi.repeat(3)


In [39]:
df_spi.inerp

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [38]:
utils.to_dekads(df_spi)

,tp,spi
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


,tp,spi
time,,
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


In [16]:
arr=np.pad(df.to_numpy().flatten()[:-1],(1,0),'constant',constant_values=(np.nan,))

In [35]:
pd.DataFrame({'sdf':arr},index=time)

,sdf
time,
2020-01-01,NaN
2020-01-11,NaN
2020-01-21,NaN
2020-02-01,-1.371
2020-02-11,-1.078
2020-02-21,-0.787
2020-03-01,-1.231
2020-03-11,-0.999
2020-03-21,-0.859
